In [1]:
import os
print('PYTHONPATH="%s"' % os.environ['PYTHONPATH'])

PYTHONPATH="/home/williams/.vscode/extensions/ms-toolsai.jupyter-2022.9.1002791758/pythonFiles:/home/williams/.vscode/extensions/ms-toolsai.jupyter-2022.9.1002791758/pythonFiles/lib/python"


In [2]:
from docplex.cp.model import CpoModel
from collections import namedtuple

#-----------------------------------------------------------------------------
# Initialize the problem data
#-----------------------------------------------------------------------------

Warehouse = namedtuple('Warehouse', ('city',      # Name of the city
                                     'capacity',  # Capacity of the warehouse
                                     'cost',      # Warehouse building cost
                                     ))

# List of warehouses
WAREHOUSES = (Warehouse("Bonn",     3, 480),
              Warehouse("Bordeaux", 1, 200),
              Warehouse("London",   2, 320),
              Warehouse("Paris",    4, 340),
              Warehouse("Rome",     1, 300))
NB_WAREHOUSES = len(WAREHOUSES)

# Number of stores
NB_STORES = 8

# Supply cost for each store and warehouse
SUPPLY_COST = ((24, 74, 31, 51, 84),
               (57, 54, 86, 61, 68),
               (57, 67, 29, 91, 71),
               (54, 54, 65, 82, 94),
               (98, 81, 16, 61, 27),
               (13, 92, 34, 94, 87),
               (54, 72, 41, 12, 78),
               (54, 64, 65, 89, 89))

/home/williams/anaconda3/lib/python3.9/site-packages/docplex-2.9.133-py3.9.egg/docplex/cp/__init__.py:30: RuntimeWarning: DOcplex.CP is supported by Python versions 2.7.9+, 3.4.x, to 3.7.x, not '3.9.12.final.0'. Use it at your own risk.
  warnings.warn(msg, RuntimeWarning)


In [3]:
#-----------------------------------------------------------------------------
# Build the model
#-----------------------------------------------------------------------------

# Create CPO model
mdl = CpoModel()

# Create one variable per store to contain the index of its supplying warehouse
NB_WAREHOUSES = len(WAREHOUSES)
supplier = mdl.integer_var_list(NB_STORES, 0, NB_WAREHOUSES - 1, "supplier")

# Create one variable per warehouse to indicate if it is open (1) or not (0)
open = mdl.integer_var_list(NB_WAREHOUSES, 0, 1, "open")

# Add constraints stating that the supplying warehouse of each store must be open
for s in supplier:
    mdl.add(mdl.element(open, s) == 1)

# Add constraints stating that the number of stores supplied by each warehouse must not exceed its capacity
for wx in range(NB_WAREHOUSES):
    mdl.add(mdl.count(supplier, wx) <= WAREHOUSES[wx].capacity)

# Build an expression that computes total cost
total_cost = mdl.scal_prod(open, [w.cost for w in WAREHOUSES])
for sx in range(NB_STORES):
    total_cost = total_cost + mdl.element(supplier[sx], SUPPLY_COST[sx])

# Minimize total cost
mdl.add(mdl.minimize(total_cost))
 

#-----------------------------------------------------------------------------
# Solve the model and display the result
#-----------------------------------------------------------------------------

# Solve model
print("\nSolving model....")
msol = mdl.solve(TimeLimit=10)

# Print solution
if msol:
    for wx in range(NB_WAREHOUSES):
        if msol[open[wx]] == 1:
            print("Warehouse '{}' open to supply stores: {}"
                  .format(WAREHOUSES[wx].city,
                          ", ".join(str(sx) for sx in range(NB_STORES) if msol[supplier[sx]] == wx)))
    print("Total cost is: {}".format(msol.get_objective_values()[0]))
else:
    print("No solution found.")


Solving model....


CpoException: Can not execute command 'cpoptimizer -angel'. Please check availability of required executable file.